# 모듈로드

In [520]:
import numpy as np
import pandas as pd
import warnings
import gc
from tqdm import tqdm_notebook as tqdm
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")
gc.enable()

In [521]:
pd.set_option('max_rows', 500)
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)

# 데이터로드

In [522]:
train_raw = pd.read_csv('./data/train.csv')
test_raw = pd.read_csv('./data/test.csv')
train_raw.shape, test_raw.shape

((200000, 202), (200000, 201))

In [523]:
bbiggu = pd.read_csv('./data_temp/bbiggu_1.csv')
bbiggu['target'] = train_raw.target

normal_idx = bbiggu[(bbiggu.target == 0) & (bbiggu.pred < 0.1)].index.values
bbiggu_idx = bbiggu[(bbiggu.target == 1) & (bbiggu.pred < 0.1)].index.values

In [524]:
train = train_raw.copy()
test = test_raw.copy()

In [525]:
col_list = train.columns[2:]

# 로직 설계

* target 0 => 110000개
* target 1 => 2600개
* 나머지 35700개에서 삐꾸 식별기 만들기

(1) 1개씩 Greedy 하게 ?

(2) 

(3)

(4)

In [589]:
train = train_raw.copy()
temp_1 = train.loc[normal_idx].copy().iloc[:100000]
temp_2 = train.loc[bbiggu_idx].copy().iloc[:2600]

dummy_1 = train.loc[normal_idx].copy().iloc[100000:]
dummy_2 = train.loc[bbiggu_idx].copy().iloc[2600:]

temp_1.target = 0
temp_2.target = 1

train = temp_1.append(temp_2)
train = train.reset_index(drop=True)

dummy = dummy_1.append(dummy_2).sort_index()
dummy.target = 1

In [592]:
target = train['target']

In [593]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):
    break

## logic

In [594]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average': False,
    'boost': 'gbdt',
    'feature_fraction_seed': 47,
    'feature_fraction': 0.041,
    'learning_rate': 0.01,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'silent': True,
    'num_threads': 8
}

In [634]:
~dummy.iloc[[i]].ID_code.isin(bbiggu[bbiggu.ID_code.isin(dummy.ID_code) & (bbiggu.bbiggu == 1)].ID_code).values[0]

True

In [635]:
num_round = 10000
root_dict = {"ID_codes":[], "iteration":[], "score":[], "index":[]}

for i in tqdm(range(dummy.shape[0])):
    
    i = i - 1
    
    if ~dummy.iloc[[i]].ID_code.isin(bbiggu[bbiggu.ID_code.isin(dummy.ID_code) & (bbiggu.bbiggu == 1)].ID_code).values[0]:
        continue
        
    
    if i == -1:
        train_new = train
        target_new = train_new.target
        
        train_index = trn_idx.copy().tolist()
        validation_index = val_idx.copy().tolist()
    else:
        new_data = dummy.iloc[[i]]

        train_new = train.append(new_data)
        target_new = train_new.target

        train_index = trn_idx.copy().tolist()
        validation_index = val_idx.copy().tolist()
        validation_index.extend(new_data.index.tolist())

    trn_data = lgb.Dataset(train_new.loc[train_index][col_list], label=target_new.loc[train_index])
    val_data = lgb.Dataset(train_new.loc[validation_index][col_list], label=target_new.loc[validation_index])

    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data],verbose_eval=False, early_stopping_rounds = 200)
    
    if i == -1:
        root_dict['ID_codes'].append('base')
        root_dict['iteration'].append(clf.best_iteration)
        root_dict['score'].append(roc_auc_score(target_new.loc[validation_index], clf.predict(train_new.loc[validation_index][col_list], num_iteration=clf.best_iteration)))
        root_dict['index'].append(i)

        del train_new, target_new, train_index, validation_index, trn_data, val_data
        gc.collect()
    else:
        root_dict['ID_codes'].append(new_data.ID_code.values[0])
        root_dict['iteration'].append(clf.best_iteration)
        root_dict['score'].append(roc_auc_score(target_new.loc[validation_index], clf.predict(train_new.loc[validation_index][col_list], num_iteration=clf.best_iteration)))
        root_dict['index'].append(i)
    
        del new_data, train_new, target_new, train_index, validation_index, trn_data, val_data
        gc.collect()

KeyboardInterrupt: 

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [627]:
a = bbiggu[bbiggu.ID_code.isin(dummy.ID_code) & (bbiggu.bbiggu == 1)]

In [636]:
pd.DataFrame(root_dict)

,ID_codes,iteration,score,index
0,train_152164,1999,0.654372,10128
1,train_152273,1999,0.654397,10201
2,train_152312,1999,0.654401,10232
3,train_152364,1999,0.654649,10271
4,train_152376,1999,0.654415,10278
5,train_152500,1999,0.653266,10364
6,train_152510,1999,0.653405,10371
7,train_152547,1999,0.653152,10399
8,train_152584,1541,0.653017,10428
9,train_152737,1999,0.654102,10539


In [638]:
t = pd.read_csv('./data_temp/dict.csv')

In [640]:
t

,ID_codes,iteration,score,index
0,base,1999,0.654277,-1
1,train_138117,1999,0.653124,0
2,train_138118,1999,0.653331,1
3,train_138119,1999,0.654783,2
4,train_138120,1999,0.653840,3
5,train_138121,1999,0.653701,4
6,train_138123,1541,0.652762,5
7,train_138126,1999,0.653240,6
8,train_138127,1999,0.654110,7
9,train_138129,1999,0.654354,8


In [639]:
t[t.iteration > 2000]

,ID_codes,iteration,score,index
243,train_138462,3054,0.665343,242
715,train_139131,3054,0.666401,714


In [612]:
idcodes = pd.DataFrame(root_dict).ID_codes

In [610]:
pd.DataFrame(root_dict).to_csv('./data_temp/dict.csv', index=False)

In [564]:
dummy.iloc[[i]].index.tolist()

[138138]

In [562]:
train_new.loc[trn_idx].head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,-5.2387,7.3746,11.5767,12.0446,11.6418,-7.0170,5.9226,-14.2136,16.0283,5.3253,12.9194,29.0460,-0.6940,5.1736,-0.7474,14.8322,11.2668,5.3822,2.0183,10.1166,16.1828,4.9590,2.0771,-0.2154,8.6748,9.5319,5.8056,22.4321,5.0109,-4.7010,21.6374,0.5663,5.1999,8.8600,43.1127,18.3816,-2.3440,23.4104,6.5199,12.1983,13.6468,13.8372,1.3675,2.9423,-4.5213,21.4669,9.3225,16.4597,7.9984,-1.7069,-21.4494,6.7806,11.0924,9.9913,14.8421,0.1812,8.9642,16.2572,2.1743,-3.4132,9.4763,13.3102,26.5376,1.4403,14.7100,6.0454,9.5426,17.1554,14.1104,24.3627,2.0323,6.7602,3.9141,-0.4851,2.5240,1.5093,2.5516,15.5752,-13.4221,7.2739,16.0094,9.7268,0.8897,0.7754,4.2218,12.0039,13.8571,-0.7338,-1.9245,15.4462,12.8287,0.3587,9.6508,6.5674,5.1726,3.1345,29.4547,31.4045,2.8279,15.6599,8.3307,-5.6011,19.0614,11.2663,8.6989,8.3694,11.5659,-16.4727,4.0288,17.9244,18.5177,10.7800,9.0056,16.6964,10.4838,1.6573,12.1749,-13.1324,17.6054,11.5423,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,10.8917,0.9003,-13.5174,2.2439,11.5283,12.0406,4.1006,-7.9078,11.1405,-5.7864,20.7477,6.8874,12.9143,19.5856,0.7268,6.4059,9.3124,6.2846,15.6372,5.8200,1.1000,9.1854,12.5963,-10.3734,0.8748,5.8042,3.7163,-1.1016,7.3667,9.8565,5.0228,-5.7828,2.3612,0.8520,6.3577,12.1719,19.7312,19.4465,4.5048,23.2378,6.3191,12.8046,7.4729,15.7811,13.3529,10.1852,5.4604,19.0773,-4.4577,9.5413,11.9052,2.1447,-22.4038,7.0883,14.1613,10.5080,14.2621,0.2647,20.4031,17.0360,1.6981,-0.0269,-0.3939,12.6317,14.8863,1.3854,15.0284,3.9995,5.3683,8.6273,14.1963,20.3882,3.2304,5.7033,4.5255,2.1929,3.1290,2.9044,1.1696,28.7632,-17.2738,2.1056,21.1613,8.9573,2.7768,-2.1746,3.6932,12.4653

In [552]:
trn_idx

array([     0,      1,      2, ..., 102596, 102597, 102599])

In [551]:
train_new.tail()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
102596,train_151577,1,6.5763,-9.2927,7.0587,4.0243,14.2409,4.0407,7.2237,13.7335,1.1738,9.3207,7.8616,-7.7491,14.1115,18.4393,6.8216,13.8723,8.2116,-13.8151,8.4051,26.5516,19.1389,10.1803,3.2025,3.5388,6.9551,14.3698,-5.9343,-3.5603,5.8303,1.1116,-19.2327,13.3207,-1.3064,20.1506,12.0015,-8.3937,0.0597,2.5207,12.6765,0.9255,-18.3639,23.8288,11.4946,11.6506,-4.4341,-34.1885,11.4974,-21.9398,15.2082,17.7783,12.6856,12.5614,0.1762,6.3621,-4.0127,7.2061,19.4519,4.9843,4.6779,8.3289,9.7394,-5.9096,1.3043,6.6146,5.9744,1.8271,5.9350,28.8586,5.0021,-5.1251,9.4000,1.1467,0.9350,15.9409,32.2042,21.8529,6.1134,22.4640,8.5298,13.3294,16.5035,12.3558,4.9601,-8.2005,-14.2161,24.2554,12.2943,15.3481,12.6156,5.7628,-0.2546,6.8502,16.0076,10.3339,8.7564,-0.4265,11.6693,13.9336,2.8640,2.0569,-28.5694,20.8398,39.0708,1.2685,12.0281,4.7551,8.1797,19.3918,14.3050,12.7187,1.3498,6.4444,2.4983,2.5000,2.9398,4.0043,2.6200,-7.9332,6.1303,4.6319,21.4555,13.0406,0.5770,0.1601,8.5217,12.2290,13.6640,2.9369,-6.4586,19.1639,11.6384,0.6320,4.9018,6.3338,-4.8333,-9.6063,43.3477,6.7191,7.3015,-5.8327,5.3020,4.8874,10.7423,8.9327,8.9809,2.1170,8.2400,-2.6857,3.9165,4.2545,15.0857,4.5542,12.1940,17.1749,16.2005,0.0400,14.0404,-13.9321,12.9819,9.8541,29.0063,5.9409,5.9035,15.6608,-2.0947,20.1795,2.7498,-12.7955,9.0331,5.6516,6.3809,-1.8596,31.8145,-4.9734,15.2135,14.6087,-12.9447,12.7950,2.5472,3.3544,-8.8933,11.4857,3.3655,6.0209,21.3709,-4.6332,10.1302,-8.9551,16.4936,-0.6171,8.5721,8.7168,0.4837,8.4766,17.6744,1.4059,-3.5900,8.6509,16.8789,-13.6006
102597,train_151649,1,6.1289,-0.9143,12.7399,6.0544,11.0810,-15.1958,5.8477,15.9536,-5.0501,7.2621,-1.0692,2.8731,14.2875,7.9075,6.5715,14.9260,7.7745,2.3625,14.5175,17.2150,6.7918,29.9404,2.7616,3.1673,6.6488,14.0225,-2.4687,-1.9853,5.5450,11.8214,-0.3965,8.9355,-2.7562,12.2900,10.7838,-1.0841,5.7148,6.8898,9.9758,4.0379,-10.2224,13.8139,10.8574,11.4719,12.4035,25.1443,10.5083,-28.4608,27.0596,27.5872,12.0777,19.3836,8.1772,7.0722,-2.8917,5.4129,14.0844,6.7769,3.6377,8.0524,8.3480,-4.6881,0.6951,-1.4784,4.4147,2.6906,5.4844,11.7506,5.0142,-1.6742,28.8856,0.7254,6.0861,28.2720,12.2195,7.9287,1.6860,10.6160,6.8057,14.3437,16.4832,14.2610,6.0781,-4.3175,6.5706,19.9106,5.8357,16.4102,5.0448,0.2243,-19.5536,6.9334,10.6517,10.7728,10.8142,-0.3631,14.8722,30.3217,1.4813,3.4290,-12.9202,17.6068,31.1300,1.6314,11.3042,3.7511,5.6434,24.4379,14.4306,27.9529,4.3833,7.1574,4.6208,6.9851,4.1955,-5.7756,0.2161,17.1450,8.4425,-2.9172,42.6729,11.1628,-4.

In [546]:
dummy[dummy.target == 1].head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
152164,train_152164,1,8.0798,1.4421,7.7231,5.3061,12.1774,-7.2362,4.5544,20.7839,-2.3368,7.0138,5.6214,-3.6104,13.9094,4.6577,5.5277,15.4997,9.3696,-9.5902,18.5468,11.1342,25.0511,17.1605,5.8220,2.4459,11.0589,13.4540,-6.1849,-2.1696,6.5751,6.1677,-15.8315,8.7565,0.1912,9.7065,10.5050,5.1990,-1.4366,7.7711,15.1837,-3.2656,-3.0826,2.8055,12.0384,11.8600,-2.8252,-12.9213,11.4665,8.2518,9.9909,28.6218,12.7814,7.3034,-7.2367,5.0884,10.8568,21.9516,19.6961,6.0380,-1.3782,8.9520,9.3637,-6.2561,3.1488,3.5782,6.7013,-0.1706,5.5357,13.3246,5.0231,-3.3629,11.0805,0.7406,2.0953,16.9251,-13.7506,30.2647,1.0636,23.2928,3.2001,13.5271,0.7789,14.8173,-1.8217,12.6964,1.1992,18.5791,5.2797,12.6361,6.7304,1.2357,-0.8029,7.2403,11.3176,11.2684,15.0826,-0.5832,0.1055,35.1438,2.0180,-1.5791,-2.9528,11.9733,7.4244,1.7767,11.4002,5.4737,8.1628,22.0780,14.0127,22.1688,10.7774,7.0761,4.3907,10.0443,1.9709,3.0760,2.9918,-0.8473,-13.0662,0.8199,29.8408,14.1915,3.6146,4.7994,6.4216,12.1988,12.9733,-2.9092,1.5833,19.6921,12.7782,0.4143,7.5247,6.8147,-3.2200,2.6964,30.1519,19.8925,-0.2937,8.8362,1.0609,-0.5087,8.0914,12.4674,8.8358,0.1825,7.7351,10.1425,4.0975,1.9108,18.3876,10.6844,7.7044,19.3180,5.2755,1.1882,11.7885,1.0773,29.1637,6.6526,36.0890,5.8176,7.3708,10.1065,3.8203,15.5952,2.9883,-14.1768,11.2927,5.7500,1.6411,0.4846,20.1197,0.3392,13.0102,10.4872,-11.9574,8.4864,6.4103,9.7912,-3.1210,10.8858,-7.6558,11.3112,5.0457,-0.0805,8.3788,-13.8439,18.0611,1.2774,6.7857,6.8344,1.3545,8.9154,22.2941,-0.5776,-0.8867,9.9084,18.0908,-1.1923
152273,train_152273,1,7.3838,-5.0633,18.0705,8.0280,10.2105,-12.3824,6.8820,15.3810,-1.1611,6.0517,-8.8538,-5.4708,13.8313,5.0771,6.1174,14.3858,8.1919,-13.9765,12.3690,24.4888,11.2618,14.0639,1.8877,3.1576,4.8780,13.9419,-3.5424,-0.5076,5.0367,1.2326,0.1833,8.6083,-1.8112,13.5426,11.6207,3.5760,2.2488,2.8734,8.0048,-2.2662,-1.8543,13.9275,10.5169,11.4657,4.5120,13.9858,13.3413,-4.6283,30.8191,34.1645,11.4717,16.6695,8.3936,6.2894,-12.0487,18.9022,11.4652,6.1339,-6.1655,10.4676,16.6034,-2.1503,3.0057,4.7720,7.7714,-5.6650,6.7400,15.1724,5.0306,0.6358,6.6776,0.0753,0.7797,17.7794,39.5983,14.6759,8.3391,19.8518,3.9454,17.2056,5.7635,15.2529,0.3206,0.7382,-10.3322,19.9459,14.3683,7.7184,11.0583,2.7529,-26.1178,7.1000,19.3064,10.6678,9.9821,-1.0803,20.6625,5.0317,1.4716,-0.1211,-5.0997,15.8110,21.4548,1.2735,10.2350,4.5052,8.5643,7.2206,14.2005,11.2053,0.2103,4.8063,4.0940,1.7148,3.0601,-0.1039,-0.4812,19.0782,0.3424,-2.5121,36.4111,9.4428,9.6774,13

In [511]:
train = train_raw.copy()
temp_1 = train.loc[normal_idx].copy().reset_index(drop=True)
temp_2 = train.loc[bbiggu_idx].copy().reset_index(drop=True)
temp_3 = temp_1.iloc[100000:100050]
temp_4 = temp_2.iloc[2500:3100]

temp_1 = temp_1.iloc[:2500]
temp_2 = temp_2.iloc[:2500]

temp_1.target = 0
temp_2.target = 1
temp_3.target = 1

train = temp_1.append(temp_2).append(temp_3)#.append(temp_4)
train = train.reset_index(drop=True)

In [512]:
train.target.value_counts()

1    2550
0    2500
Name: target, dtype: int64

In [513]:
train.tail()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
5045,train_138178,1,4.7222,-6.5607,12.7414,10.7398,9.8036,-13.4954,6.3972,17.1144,-3.4170,6.8393,-11.0083,-7.2956,13.9320,12.9577,4.1975,14.3070,8.7752,-6.2193,27.2111,8.5936,11.1990,22.4830,2.2544,3.9465,11.4570,13.6128,-12.8651,-2.3583,6.7071,3.7255,-4.2776,8.4849,-1.6638,9.3282,11.8092,1.9516,0.3264,7.0073,6.4881,3.7484,3.3969,23.7103,12.2952,11.1853,19.3805,17.5900,13.4870,1.1742,2.5975,9.1361,12.3316,9.4293,-7.4846,5.0200,-0.2612,5.6703,18.8020,7.0088,2.6174,9.9541,6.1927,-15.5297,-1.4898,-2.1063,7.4961,-0.0577,6.3438,31.0242,5.0236,0.4833,33.9955,0.3957,-6.1819,9.1596,-6.2562,21.8667,8.5619,21.0451,4.1680,13.3981,2.9557,15.3562,2.2393,9.4649,-9.6052,11.5660,15.7281,1.9839,8.6876,8.9964,-8.7663,6.8815,12.8538,10.0849,14.4006,0.2452,5.3330,32.8350,2.0087,-1.5448,-13.1893,18.3891,0.1731,1.2720,8.7964,3.5414,9.2645,12.8583,14.4898,15.8940,4.0356,6.4597,5.7263,6.2884,3.9104,4.1657,3.2020,11.7367,2.8704,-1.7354,41.0909,10.0676,-2.5246,5.4644,7.4987,12.6103,12.4785,0.4783,-5.2411,7.9304,12.1119,-0.1981,9.7289,6.8875,-5.0241,-5.1362,16.2673,6.5711,5.3739,2.4883,6.7125,4.8339,11.9066,10.5039,9.3287,1.8299,11.8195,-2.6244,3.8658,-4.3155,16.1019,3.2515,6.0872,15.7237,17.1726,-11.6236,13.4329,-0.3809,25.2940,7.5328,17.7056,5.4026,6.2048,19.2789,3.5933,23.9582,2.4143,-2.9056,5.5566,5.1570,1.0791,-5.2347,22.6354,-0.0001,14.1219,14.3546,2.8016,10.7790,-2.5700,3.1252,-6.9395,13.7118,3.2319,4.8317,-2.3609,-10.7237,6.8330,0.9565,17.2911,1.7357,5.1976,4.8373,1.7556,11.0084,18.7561,-0.6506,-2.0771,10.6484,20.4937,-2.4981
5046,train_138179,1,9.8485,-0.7884,12.1896,8.9858,11.9194,-14.8527,6.0571,16.2003,5.5767,8.7277,0.9658,-12.2154,13.9340,11.9280,4.8543,14.2752,11.1435,-7.4627,11.6810,25.4261,25.7691,22.4857,4.1029,3.4745,12.3614,14.0323,11.1886,-2.9525,6.5478,0.7201,-11.9535,10.8357,-0.6388,14.6646,10.8762,8.7063,8.5519,6.1109,14.3119,8.8851,-15.2695,13.2119,11.7721,11.3798,9.8293,-12.2583,9.8715,-17.7773,22.4313,22.4494,13.1501,17.1783,0.4875,6.6579,5.7704,16.8117,19.5838,6.0221,0.3114,8.1948,9.8983,-19.7468,0.5933,-3.2023,4.9432,-1.6960,5.4379,0.5261,5.0189,-1.7481,22.3265,0.5462,2.1665,18.9280,37.0709,10.0538,6.5298,24.1485,6.7379,16.5610,20.2177,13.1715,14.2932,10.8245,-13.7896,16.5193,4.7796,19.9865,5.8611,4.8611,-25.6038,7.1146,18.4685,10.2136,8.1837,-0.6556,10.5909,25.3144,1.2317,-4.5470,-15.8986,12.1479,16.5475,1.5558,9.4734,2.7289,7.4516,23.6273,14.0218,11.2572,8.6164,7.1112,3.3870,4.5346,2.9935,3.4537,4.5943,36.9857,10.9801,3.4561,33.1245,12.9772,9.8419

In [400]:
target = train['target']

In [383]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average': False,
    'boost': 'gbdt',
    'feature_fraction_seed': 47,
    'feature_fraction': 0.041,
    'learning_rate': 0.01,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1,
    'num_threads': 8
}

In [505]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_lgb = np.zeros(len(train))
# predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [c for c in train.columns if c not in ['ID_code', 'target']]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target.values)):
    break
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=400, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
#     predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
    print("CV score: {:<8.5f}".format(roc_auc_score(target.values[val_idx], oof_lgb[val_idx])))
    
    break
    
# print("CV score: {:<8.5f}".format(roc_auc_score(target.values, oof_lgb)))

In [514]:
target = train['target']

In [515]:
train_index = trn_idx.tolist()
validation_index = val_idx.tolist()

In [516]:
validation_index.extend(train.index[-50:].tolist())

In [517]:
len(train_index)

4000

In [518]:
len(validation_index)

1050

In [519]:
trn_data = lgb.Dataset(train.iloc[train_index][train_columns], label=target.iloc[train_index])
val_data = lgb.Dataset(train.iloc[validation_index][train_columns], label=target.iloc[validation_index])

num_round = 10000
clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=400, early_stopping_rounds = 200)
oof_lgb[validation_index] = clf.predict(train.iloc[validation_index][train_columns], num_iteration=clf.best_iteration)

fold_importance = pd.DataFrame()
fold_importance["Feature"] = train_columns
fold_importance["importance"] = clf.feature_importance()
fold_importance["fold"] = fold_ + 1
feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

print("CV score: {:<8.5f}".format(roc_auc_score(target.values[validation_index], oof_lgb[validation_index])))

fold n°0
Training until validation scores don't improve for 200 rounds.
[400]	training's auc: 0.952282	valid_1's auc: 0.595887
[800]	training's auc: 0.978332	valid_1's auc: 0.608262
Early stopping, best iteration is:
[608]	training's auc: 0.969965	valid_1's auc: 0.609156
CV score: 0.60916 
